In [1]:
from openff.toolkit import Molecule
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper
from openff.nagl_models import get_models_by_type, validate_nagl_model_path

In [2]:
# Grab the latest production NAGL model
nagl_model = get_models_by_type(model_type='am1bcc', production_only=True)[-1]
model_path = validate_nagl_model_path(nagl_model)

In [3]:
my_mol = Molecule.from_file("../polymer/HPMCAS-H-Trimer.sdf")
# Note there are no partial charges
print(my_mol.partial_charges)
# Now use NAGL to assign the partial charges
my_mol.assign_partial_charges(toolkit_registry=NAGLToolkitWrapper(), partial_charge_method=model_path,)

None


In [4]:
my_mol.partial_charges

Magnitude,[0.4064335569000996 0.09678563915260203 0.10430550269849666 0.33410937033660776 -0.39855443991653555 0.12056348167426952 0.0927927761531628 0.12040539525516399 -0.5794452384852611 -0.41596185721389883 -0.3887449160956585 -0.4293078438185894 0.6533923789120473 -0.157144288792535 -0.5304586247348033 0.11735260181196101 0.12987707994945413 0.13856898687847025 -0.11882370270721547 -0.5932327464961253 0.06521797619827159 0.10682997547157176 0.045508500859812576 0.07587670229442485 0.05800582163102992 0.04822634972699054 0.04822634972699054 0.08127680860526927 0.08127680860526927 0.08127680860526927 0.03580890663035281 0.03580890663035281 0.03580890663035281 0.043408651576117355 0.043408651576117355 0.048085977063254196 0.047268108204678375 0.047268108204678375 0.047268108204678375 0.39974514089591867 0.08478558503158458 0.09007808260686763 0.334713284597472 -0.3993812457465374 0.11276406489379771 0.13345483385093576 0.1347587748623646 -0.3755864159011089 -0.4386219934367382 -0.3749967173480236 -0.3758794919394695 0.12620906971462137 0.1164407476044453 0.647476558313445 -0.5459101156138622 -0.13374353505603903 -0.1238038048886501 0.6344623609638966 -0.5445361093424999 -0.6059878424548351 0.052147623822764236 0.06163763694532283 0.04685935072667964 0.06692777268655666 0.06581393323905833 0.07007218591936 0.07007218591936 0.032167476341799575 0.032167476341799575 0.032167476341799575 0.035423022434786636 0.035423022434786636 0.035423022434786636 0.10001022063262828 0.10001022063262828 0.10050895504720576 0.10050895504720576 0.4492487355328358 0.1124392940855778 0.07780410789020427 0.10651728950508006 -0.385404701604768 0.0920589014149464 0.14736114881999857 0.13479418002136118 -0.3744223967932903 -0.4433115080737316 -0.38101043857566946 -0.39595800198547476 0.12130880795486339 0.11587177656658061 0.6419429226971425 -0.5394556478404247 -0.1521878794573986 0.062185846940354186 0.05021984942324527 0.0620530887699879 0.07588701389797099 0.0639156147099293 0.07077576659686931 0.07077576659686931 0.033205245361403304 0.033205245361403304 0.033205245361403304 0.035434768275097686 0.035434768275097686 0.035434768275097686 0.07868865571983226 0.07868868552215465 0.07868868552215465 0.0620530887699879]
Units,elementary_charge


In [5]:
my_mol.name

'HPMCAS-H-Trimer-polymer'

In [6]:
from openpharmmdflow.pipeline.sm.pipeline_settings import *
from openpharmmdflow.pipeline.sm.pipeline import SmallMoleculePipeline
import nglview

In [7]:
settings = {
    # Not used right now
    "work_dir": "/home/mmh/Projects/OpenPharmMDFlow/experiments/sm/scratch",
    "inputs": [my_mol],
    "prep_config": SmallMoleculePipelinePrepConfig(
        bespokefit_config=BespokefitConfig(
            bespoke_workflow_factory_config=BespokeWorkflowFactoryConfig(),
            bespoke_executor_config=BespokeExecutorConfig(),
            mol_to_bespoke="ibuprofen",
        )
    ),
    "pack_config": SmallMoleculePipelinePackConfig(
        molecule_names=[my_mol.name,],
        number_of_copies=[5],
        target_density=0.1,
    ),
    "parameterize_config": SmallMoleculePipelineParameterizeConfig(),
    "simulate_config": SmallMoleculePipelineSimulateConfig(),
    "analyize_config": SmallMoleculePipelineAnalyizeConfig(),
    "solvate_config": SmallMoleculePipelineSolvateConfig(
    nacl_conc=Quantity(0.1, "mole / liter"),
    padding=Quantity(1.2, "nanometer"),
    box_shape=RHOMBIC_DODECAHEDRON,
    target_density=Quantity(0.9, "gram / milliliter"),
    tolerance=Quantity(2.0, "angstrom"),
    )
    }
sm_config = SmallMoleculePipelineConfig(**settings)

In [8]:
smp = SmallMoleculePipeline(sm_config)
smp.load()

In [9]:
smp.pack()

In [10]:
smp.topology.to_file("system.pdb")
nglview.show_structure_file("system.pdb")

NGLWidget()

In [11]:
smp.solvate()

In [12]:
smp.solvated_topology.to_file("system.pdb")
nglview.show_structure_file("system.pdb")

NGLWidget()

In [13]:
smp.parameterize()

/home/mmh/micromamba/envs/openpharmmdflow-new/lib/python3.12/site-packages/openff/interchange/components/interchange.py:977: UserWarning: Interchange object combination is experimental and likely to produce strange results. Any workflow using this method is not guaranteed to be suitable for production. Use with extreme caution and thoroughly validate results!
  return _combine(self, other)
/home/mmh/micromamba/envs/openpharmmdflow-new/lib/python3.12/site-packages/openff/interchange/operations/_combine.py:126: UserWarning: Setting positions to None because one or both objects added together were missing positions.
  warnings.warn(


In [14]:
smp.interchange.visualize("nglview")

NGLWidget()

In [15]:
smp.simulate()

start minimize
end minimize
Starting simulation
Step, volume (nm^3)
0 157.025
500 156.041
1000 152.755
1500 151.761
2000 149.976
2500 148.864
3000 148.148
3500 147.539
4000 146.61
4500 145.584
Elapsed time: 5.94 seconds
